## 1. 데이터 준비와 전처리

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies = movies.set_index('movie_id')
print(movies)

                                       title                         genre
movie_id                                                                  
1                           Toy Story (1995)   Animation|Children's|Comedy
2                             Jumanji (1995)  Adventure|Children's|Fantasy
3                    Grumpier Old Men (1995)                Comedy|Romance
4                   Waiting to Exhale (1995)                  Comedy|Drama
5         Father of the Bride Part II (1995)                        Comedy
...                                      ...                           ...
3948                 Meet the Parents (2000)                        Comedy
3949              Requiem for a Dream (2000)                         Drama
3950                        Tigerland (2000)                         Drama
3951                 Two Family House (2000)                         Drama
3952                   Contender, The (2000)                Drama|Thriller

[3883 rows x 2 columns]


In [7]:
ratings = ratings.join(movies, on='movie_id')
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [8]:
# 관사가 모두 뒤로 빠져있어 헷갈리므로 제거
ratings['title'] = ratings['title'].str.replace(', The','')
ratings['title'] = ratings['title'].str.replace(', A','')

## 2. 데이터 분석

In [9]:
ratings['movie_id'].nunique() # ratings에 있는 유니크한 영화 개수

3628

In [10]:
ratings['user_id'].nunique() # ratings에 있는 유니크한 사용자 수

6039

In [16]:
# 가장 인기 있는 영화 30개(인기순)
movie_count = ratings.groupby('title')['movie_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs (1991)                              2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix (1999)                                            2434
Jurassic Park (1993)                                     2413
Sixth Sense (1999)                                       2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [17]:
my_favorite = ['Matrix (1999)' , 'Gladiator (2000)' ,'Toy Story (1995)' ,'Usual Suspects (1995)' ,'Braveheart (1995)']

my_movielist = pd.DataFrame({'user_id': ['ws']*5, 'title': my_favorite, 'count': [5]*5})

if not ratings.isin({'user_id':['ws']})['user_id'].any():  
    ratings = ratings.append(my_movielist)                         

ratings.tail(10)      

,user_id,movie_id,count,timestamp,title,genre
1000203,6040,1090.0,3,956715518.0,Platoon (1986),Drama|War
1000205,6040,1094.0,5,956704887.0,Crying Game (1992),Drama|Romance|War
1000206,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096.0,4,956715648.0,Sophie's Choice (1982),Drama
1000208,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,ws,NaN,5,NaN,Matrix (1999),NaN
1,ws,NaN,5,NaN,Gladiator (2000),NaN
2,ws,NaN,5,NaN,Toy Story (1995),NaN
3,ws,NaN,5,NaN,Usual Suspects (1995),NaN
4,ws,NaN,5,NaN,Braveheart (1995),NaN


## 4. CSR matrix

* indexing

In [18]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['ws'])   # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다.  
print(movie_to_idx['Matrix (1999)'])

6039
124


In [19]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings): # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,1193.0,5,978300760.0,0,Drama
1,0,661.0,3,978302109.0,1,Animation|Children's|Musical
2,0,914.0,3,978301968.0,2,Musical|Romance
3,0,3408.0,4,978300275.0,3,Drama
4,0,2355.0,5,978824291.0,4,Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,NaN,5,NaN,124,NaN
1,6039,NaN,5,NaN,67,NaN
2,6039,NaN,5,NaN,40,NaN
3,6039,NaN,5,NaN,233,NaN


* CSR matrix 만들기

In [20]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. als_model = AlternatingLeastSquares 모델 훈련

In [21]:
from implicit.als import AlternatingLeastSquares
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [22]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [23]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [24]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6. 훈련 모델이 예측한 나의 선호도 파악

In [27]:
# 내가 선호하는 영화
ws, Matrix = user_to_idx['ws'], movie_to_idx['Matrix (1999)']
ws_vector, Matrix_vector = als_model.user_factors[ws], als_model.item_factors[Matrix]

ws_vector

array([ 0.04075857,  0.3744514 , -0.27003548,  0.41674805,  0.2536449 ,
       -0.1908198 , -1.0303776 , -0.8233736 ,  0.06573096,  0.52096367,
       -0.61130357,  0.6535155 ,  0.2302155 ,  0.6183274 ,  0.16697983,
       -0.600277  , -0.45353276, -0.26871783,  0.3179338 ,  0.00967019,
        0.74171233, -0.13740362, -0.4608095 ,  0.28155845, -0.1620554 ,
        0.4192723 , -0.16297606, -0.02687179, -0.33476195,  0.19409572,
        0.05318561,  0.87604123,  0.27230695, -0.14381406, -0.79486763,
       -0.8004102 , -0.10094889,  0.3311126 ,  0.02353918,  0.59430426,
        0.25930816,  0.42931125, -0.66051257, -0.27254266,  0.15132022,
       -1.0191859 ,  0.15146452, -0.37466997,  0.537797  ,  0.7028465 ,
       -0.66290563,  0.22502647,  0.18520996, -0.70441103,  1.63987   ,
       -0.16154376, -0.6186564 , -0.5279434 , -0.19384474, -0.22665864,
       -0.05212646,  0.8372588 , -0.8345463 ,  0.1917867 ,  1.0944096 ,
        0.06936143, -0.02084614,  0.4036667 ,  0.08575428, -0.04

In [28]:
Matrix_vector

array([ 0.01352236,  0.02332378, -0.02199415,  0.02541124,  0.01201019,
       -0.00259323, -0.02686229, -0.01125575,  0.02049573,  0.00938175,
        0.00633568,  0.04389724, -0.00011668, -0.00781899,  0.00927566,
       -0.00032802, -0.0248587 ,  0.03101616,  0.01432049, -0.00808233,
        0.01513345,  0.00896645, -0.01917361,  0.01756753,  0.00060591,
        0.01599445, -0.00323325, -0.00052188,  0.00593243,  0.00269389,
       -0.00646979,  0.0049224 ,  0.01537833, -0.00889848, -0.0030751 ,
       -0.00967646, -0.00240522, -0.00783247,  0.01215429,  0.04126461,
        0.00486668,  0.0117424 , -0.01825899, -0.00783046, -0.00416893,
        0.01587504,  0.00744839, -0.01390404, -0.00386901,  0.01555301,
       -0.01578207,  0.01832595,  0.00641469, -0.0434703 ,  0.04071029,
        0.00742438, -0.01708283,  0.0193011 , -0.01103148, -0.00612076,
       -0.00707412, -0.00996691, -0.0056034 ,  0.04474783,  0.02464409,
        0.0038813 ,  0.01746648,  0.01379651, -0.00305058, -0.01

In [29]:
# ws와 Matrix를 내적하는 코드
np.dot(ws_vector, Matrix_vector)

0.5087195

In [30]:
# 다른 영화에 대해 어떻게 예측할지 확인
Godfather = movie_to_idx['Godfather (1972)']
Godfather_vector = als_model.item_factors[Godfather]
np.dot(ws_vector, Godfather_vector) 

0.06555245

## 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

* AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 사용

In [49]:
favorite_movie = 'Matrix (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(124, 0.99999994),
 (92, 0.8096528),
 (62, 0.68368053),
 (200, 0.57985634),
 (141, 0.55936265),
 (107, 0.55536634),
 (145, 0.5269561),
 (175, 0.49107453),
 (375, 0.48017824),
 (75, 0.46238944),
 (44, 0.4351607),
 (317, 0.42604202),
 (117, 0.40263215),
 (236, 0.38308364),
 (3156, 0.38102725)]

In [32]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Matrix (1999)',
 'Terminator 2: Judgment Day (1991)',
 'Total Recall (1990)',
 'Terminator (1984)',
 'Fugitive (1993)',
 'Jurassic Park (1993)',
 'Fifth Element (1997)',
 'Men in Black (1997)',
 'Face/Off (1997)',
 'Hunt for Red October (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Twelve Monkeys (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Speed (1994)',
 'Running Free (2000)']

In [33]:
# 위의 코드를 함수로 만들기
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id, N=15)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [34]:
 get_similar_movie('Gladiator (2000)')

['Gladiator (2000)',
 'Mission: Impossible 2 (2000)',
 'Patriot (2000)',
 'Perfect Storm (2000)',
 'X-Men (2000)',
 'U-571 (2000)',
 'Gone in 60 Seconds (2000)',
 'Frequency (2000)',
 'Shanghai Noon (2000)',
 'Romeo Must Die (2000)',
 'Fight Club (1999)',
 'Hamlet (2000)',
 'Rules of Engagement (2000)',
 'Chicken Run (2000)',
 'Me, Myself and Irene (2000)']

* 비슷한 영화 목록이 이상하다. 다른 것들도 확인해본다.

In [35]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life (1998)",
 'Aladdin (1992)',
 'Babe (1995)',
 'Groundhog Day (1993)',
 "There's Something About Mary (1998)",
 'Pleasantville (1998)',
 'Lion King (1994)',
 'Beauty and the Beast (1991)',
 'Shakespeare in Love (1998)',
 'Hercules (1997)',
 'Santa Clause (1994)',
 'Full Monty (1997)',
 'Forrest Gump (1994)']

In [36]:
get_similar_movie('Braveheart (1995)')

['Braveheart (1995)',
 'Saving Private Ryan (1998)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Last of the Mohicans (1992)',
 'Boat (Das Boot) (1981)',
 'Men in Black (1997)',
 'Legends of the Fall (1994)',
 'Boogie Nights (1997)',
 'Hunt for Red October (1990)',
 'Fugitive (1993)',
 'Make Them Die Slowly (Cannibal Ferox) (1980)',
 'Matrix (1999)']

In [37]:
get_similar_movie('Usual Suspects (1995)')

['Usual Suspects (1995)',
 'Reservoir Dogs (1992)',
 'Pulp Fiction (1994)',
 'L.A. Confidential (1997)',
 'Seven (Se7en) (1995)',
 'Fargo (1996)',
 'GoodFellas (1990)',
 'Silence of the Lambs (1991)',
 'Simple Plan (1998)',
 'Shawshank Redemption (1994)',
 'Sixth Sense (1999)',
 'Condition Red (1995)',
 'Sling Blade (1996)',
 'Professional (a.k.a. Leon: The Professional) (1994)',
 'Lock, Stock & Two Smoking Barrels (1998)']

* 다른 것들은 좀 더 낫다.

## 8.  내가 가장 좋아할 만한 영화들을 추천받기

In [39]:
user = user_to_idx['ws']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(48, 0.50075936),
 (92, 0.4856974),
 (141, 0.3935561),
 (224, 0.38651264),
 (346, 0.37076536),
 (38, 0.36830103),
 (50, 0.3499253),
 (75, 0.3150213),
 (482, 0.30207288),
 (372, 0.30157542),
 (121, 0.29500484),
 (222, 0.29246774),
 (157, 0.2830846),
 (107, 0.26517424),
 (248, 0.2490945),
 (289, 0.24212255),
 (220, 0.23789686),
 (4, 0.23553193),
 (126, 0.233955),
 (51, 0.23212096)]

In [41]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Saving Private Ryan (1998)',
 'Terminator 2: Judgment Day (1991)',
 'Fugitive (1993)',
 'L.A. Confidential (1997)',
 'Patriot (2000)',
 'Sixth Sense (1999)',
 'Toy Story 2 (1999)',
 'Hunt for Red October (1990)',
 'Mission: Impossible 2 (2000)',
 'X-Men (2000)',
 'Silence of the Lambs (1991)',
 'Pulp Fiction (1994)',
 'Shawshank Redemption (1994)',
 'Jurassic Park (1993)',
 'Good Will Hunting (1997)',
 'Reservoir Dogs (1992)',
 'Seven (Se7en) (1995)',
 "Bug's Life (1998)",
 'Shakespeare in Love (1998)',
 'Fargo (1996)']

In [44]:
# 추천 기여도 확인
spr = movie_to_idx['Saving Private Ryan (1998)']
explain = als_model.explain(user, csr_data, itemid=spr)# explain : 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Braveheart (1995)', 0.3475042015833578),
 ('Toy Story (1995)', 0.058579406838086254),
 ('Matrix (1999)', 0.05232497148921312),
 ('Gladiator (2000)', 0.04453268743278063),
 ('Usual Suspects (1995)', -0.009455883887337785)]

In [45]:
t2 = movie_to_idx['Terminator 2: Judgment Day (1991)']
explain = als_model.explain(user, csr_data, itemid=t2)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Matrix (1999)', 0.3004299989204403),
 ('Braveheart (1995)', 0.1387482798241208),
 ('Usual Suspects (1995)', 0.031858270749079695),
 ('Toy Story (1995)', 0.012462678282483628),
 ('Gladiator (2000)', -0.005094875000178693)]

In [46]:
pa = movie_to_idx['Patriot (2000)']
explain = als_model.explain(user, csr_data, itemid=pa)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Gladiator (2000)', 0.26696086200192193),
 ('Braveheart (1995)', 0.06503277283051585),
 ('Matrix (1999)', 0.029276177893240354),
 ('Toy Story (1995)', 0.01054455097292193),
 ('Usual Suspects (1995)', -0.007100797327211331)]

In [47]:
xm = movie_to_idx['X-Men (2000)']
explain = als_model.explain(user, csr_data, itemid=xm)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Gladiator (2000)', 0.20021257297990228),
 ('Matrix (1999)', 0.13218660735908516),
 ('Toy Story (1995)', 0.004156298953612303),
 ('Usual Suspects (1995)', -0.011053484304764556),
 ('Braveheart (1995)', -0.027343437788878173)]

* 'Gladiator'의 유사영화 추천 목록이 문제가 있었으므로, 그 기여도가 컸을 영화까지 확인해보았다.
* 이 영화와 유사한 영화가 자료에 별로 없었던 것 같다. 아래에서 확인해본다.

In [51]:
favorite_movie = 'Gladiator (2000)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(67, 1.0000001),
 (482, 0.6445144),
 (346, 0.62556595),
 (746, 0.54905796),
 (372, 0.48011428),
 (1537, 0.47944364),
 (327, 0.4773962),
 (463, 0.44334835),
 (300, 0.44274238),
 (1737, 0.37012982),
 (273, 0.35390022),
 (2410, 0.35041845),
 (464, 0.34107658),
 (474, 0.3372565),
 (1027, 0.33427197)]

## 회고

* 평소 추천 시스템을 반신반의 하는지라 시작할 때 그리 많은 관심이 가지는 않았다. 결과는 딱 이 지점으로 회귀했다.
* 2000년 작 글래디에이터의 유사영화 추천 목록은 의심을 증폭시켰다. 유사한 것으로 보이는 영화들도 있었지만, 개봉연도가 같은 것을 제외하면 유사점이 잘 안 보이는 것들도 있었다.
* 선호 영화 다섯개를 기반으로 한 영화 추천 목록과 그 추천에 대한 각각 영화의 기여도를 몇 개 확인해보자 그 의심이 조금 풀렸다. 우선 글래디에이터 유사영화에서 전혀 뜬금 없던 것은 걸러졌다. 다른 영화들의 영향으로 보인다.
* 글래디에이터의 기여가 가장 컸을 것으로 생각되는 추천작 패트리어트(2000)와 엑스맨(2000)의 기여도를 확인해보니 다른 것들에 비해 기여도가 큰 영화의 영향력이 적다. 특히 유사목록에는 있었지만 개봉연도 빼고는 유사점이 없었던 엑스맨은 매트릭스의 영향을 많이 받아 추천작에 포함 된 것으로 보인다. 
* 위와 같은 결과는 데이터에 글래디에이터와 유사한 영화가 별로 없었기 때문에 그나마 유사한 개봉연도가 유사도에 많이 기여한 것이 아닌가 한다. 실제로 이 영화의 유사작들의 유사도를 확인해 본 결과 유사 목록의 유사함이 나았던 매트릭스에 비해 유사도가 다소 떨어지는 것이 확인되었다.
* 추천시스템을 유의미하게 사용하려면, 많은 데이터를 확보하는 것이 유의미해 보이며, 상호보완해야 함을 알 수 있었다.
* 또한 유저의 니즈를 좀 더 파악하고 아이템의 성격을 더 잘 알아야 할 것 같다. 이를테면 상위에 추천된 2000년 작 패트리어트의 경우, 거대한 힘에 맞선 영웅의 이야기라는 점에서 글래디에이터와 비슷한 얼개를 가진 작품이지만, 각색이 심하다 못해 역사왜곡에 이른 작품임을 알기에 선뜻 선택하기 힘든 작품이었다. 이런 점들도 모두 데이터라는 점에서 역시 모델보다는 데이터가 중요하다는 것을 다시 한 번 느낀다. 